In [2]:
pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [24]:
import pandas as pd
import numpy as np
import pydicom
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load CSV files
train_labels = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/data/final_data/train.csv')
train_coords = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/data/final_data/train_label_coordinates.csv')

def load_dicom_image(file_path):
    dicom = pydicom.dcmread(file_path)  # DICOM 파일을 읽어들임
    image = dicom.pixel_array  # DICOM 파일에서 이미지 데이터 추출
    image = cv2.resize(image, (128, 128))  # 이미지를 64x64 크기로 조정
    image = image / np.max(image)  # 픽셀 값을 0과 1 사이로 정규화
    return image  # 정규화된 이미지를 반환

In [23]:
train_labels.isnull().sum()

study_id                                    0
spinal_canal_stenosis_l1_l2                 1
spinal_canal_stenosis_l2_l3                 1
spinal_canal_stenosis_l3_l4                 1
spinal_canal_stenosis_l4_l5                 1
spinal_canal_stenosis_l5_s1                 1
left_neural_foraminal_narrowing_l1_l2       2
left_neural_foraminal_narrowing_l2_l3       2
left_neural_foraminal_narrowing_l3_l4       2
left_neural_foraminal_narrowing_l4_l5       2
left_neural_foraminal_narrowing_l5_s1       2
right_neural_foraminal_narrowing_l1_l2      8
right_neural_foraminal_narrowing_l2_l3      8
right_neural_foraminal_narrowing_l3_l4      8
right_neural_foraminal_narrowing_l4_l5      8
right_neural_foraminal_narrowing_l5_s1      8
left_subarticular_stenosis_l1_l2          164
left_subarticular_stenosis_l2_l3           82
left_subarticular_stenosis_l3_l4            3
left_subarticular_stenosis_l4_l5            3
left_subarticular_stenosis_l5_s1           11
right_subarticular_stenosis_l1_l2 

In [21]:
train_labels[train_labels.study_id==3008676218]

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1378,3008676218,NaN,NaN,NaN,NaN,NaN,Normal/Mild,Severe,Moderate,Moderate,...,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Severe,Normal/Mild,Normal/Mild


In [27]:
train_coords[train_coords.study_id==3008676218]

,study_id,series_id,instance_number,condition,level,x,y


In [31]:

# def normalize_condition(condition): # 컨디션으로 넘어온 문자열에서_를 공백문자로 바꾼후 문자열 첫 글자를 대문자로 치환
#     return condition.replace('_', ' ').title()

# def normalize_level(level): #level 변수의 _를 /으로 하고 대문자로 치환
#     return level.replace('_', '/').upper()

# images = []  # 이미지를 저장할 리스트
# labels = []  # 레이블을 저장할 리스트

# # train_labels 데이터프레임의 각 행에 대해 반복
# for _, row in train_labels.iterrows():
#     study_id = row['study_id']  # 각 행의 study_id 추출 	4003253

# # 각 행의 컬럼에 대해 반복 (첫 번째 컬럼은 study_id이므로 무시)
#     for col in train_labels.columns[1:]:
#         condition_level = col.split('_')  # 컬럼명을 condition과 level로 분리
#         #print(condition_level)
#         condition = '_'.join(condition_level[:-2])  # condition 부분 추출
#         #print(condition)
#         level = '_'.join(condition_level[-2:])  # level 부분 추출
#         #print(level)
#         severity = row[col]  # 각 열에 대한 심각도(severity) 추출
#         #print(severity)
#         # 심각도(severity)가 NaN인 경우 건너뜀
#         if pd.isna(severity):
#             continue

#     # condition과 level을 정규화하여 일치시킴
#         normalized_condition = normalize_condition(condition)
#         normalized_level = normalize_level(level)

#     # 해당 조건(condition), 레벨(level), study_id에 대한 좌표를 찾음
#         coords = train_coords[(train_coords['study_id'] == study_id) &
#                                     (train_coords['condition'] == normalized_condition) &
#                                     (train_coords['level'] == normalized_level)]
#         print(coords.iloc[1])

In [29]:
def normalize_condition(condition): # 컨디션으로 넘어온 문자열에서_를 공백문자로 바꾼후 문자열 첫 글자를 대문자로 치환
    return condition.replace('_', ' ').title()

def normalize_level(level): #level 변수의 _를 /으로 하고 대문자로 치환
    return level.replace('_', '/').upper()

# 데이터 준비 함수
def prepare_data(train_labels, train_coords, image_dir):
    images = []  # 이미지가 순서대로 np형식으로 저장
    labels = []  # 각각의 심각도 수준이 순서대로 np형식으로 저장

    # train_labels 데이터프레임의 각 행에 대해 반복
    for _, row in train_labels.iterrows():
        study_id = row['study_id']  # 각 행의 study_id 추출 	4003253

        # 각 행의 컬럼에 대해 반복 (첫 번째 컬럼은 study_id이므로 무시)
        for col in train_labels.columns[1:]:
            condition_level = col.split('_')  # 컬럼명을 condition과 level로 분리
            condition = '_'.join(condition_level[:-2])  # condition 부분 추출
            level = '_'.join(condition_level[-2:])  # level 부분 추출
            severity = row[col]  # 해당 조건에 대한 심각도(severity) 추출

            # 심각도(severity)가 NaN인 경우 건너뜀
            if pd.isna(severity):
                continue

            # condition과 level을 정규화하여 일치시킴
            normalized_condition = normalize_condition(condition)
            normalized_level = normalize_level(level)

            # 해당 조건(condition), 레벨(level), study_id가 같은 train_coords 행을 찾음
            coords = train_coords[(train_coords['study_id'] == study_id) &
                                  (train_coords['condition'] == normalized_condition) &
                                  (train_coords['level'] == normalized_level)]

            if not coords.empty:
                series_id = coords.iloc[0]['series_id']  # 일치하는 행의 series_id 추출
                instance_number = coords.iloc[0]['instance_number']  # 일치하는 행의 instance_number 추출
                img_path = f'{image_dir}/{study_id}/{series_id}/{instance_number}.dcm'  # DICOM 이미지 경로 생성

                if os.path.exists(img_path):
                    image = load_dicom_image(img_path)  # DICOM 이미지를 로드
                    images.append(image)  # 이미지 리스트에 추가
                    labels.append(severity)  # 레이블 리스트에 추가
                else:
                    print(f'Image not found: {img_path}')  # 이미지를 찾을 수 없는 경우 메시지 출력
            else:
                print(f'No coordinates found for {study_id}, {normalized_condition}, {normalized_level}')  # coords가 비었다면  메시지 출력

    return np.array(images), np.array(labels)  # 리스트를 넘파이 배열로 변환하여 반환

In [30]:
# Example: Preparing data
image_dir = '/content/drive/MyDrive/멀티캠퍼스/data/final_data/train_images'  # Update with correct path
X, y = prepare_data(train_labels, train_coords, image_dir)

# Print the shapes of X and y to verify data loading
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

No coordinates found for 74782131, Spinal Canal Stenosis, L1/L2
No coordinates found for 267842058, Spinal Canal Stenosis, L1/L2
No coordinates found for 267989673, Spinal Canal Stenosis, L1/L2
No coordinates found for 293713262, Spinal Canal Stenosis, L1/L2
No coordinates found for 293713262, Spinal Canal Stenosis, L2/L3
No coordinates found for 296083289, Spinal Canal Stenosis, L1/L2
No coordinates found for 305152236, Spinal Canal Stenosis, L1/L2
No coordinates found for 344297746, Spinal Canal Stenosis, L1/L2
No coordinates found for 376723024, Spinal Canal Stenosis, L1/L2
No coordinates found for 376723024, Spinal Canal Stenosis, L2/L3
No coordinates found for 390498354, Spinal Canal Stenosis, L1/L2
No coordinates found for 390498354, Spinal Canal Stenosis, L2/L3
No coordinates found for 434488359, Spinal Canal Stenosis, L1/L2
No coordinates found for 434488359, Spinal Canal Stenosis, L2/L3
No coordinates found for 597329259, Spinal Canal Stenosis, L1/L2
No coordinates found for 6

KeyboardInterrupt: 

In [ ]:
# Check if the dataset is empty before proceeding
if X.size > 0 and y.size > 0:
    # Encode labels
    severity_mapping = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}
    y_encoded = np.array([severity_mapping[severity] for severity in y])
    y_categorical = to_categorical(y_encoded, num_classes=3)

    # Train-test split
    X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

    # Add a channel dimension to the images
    X_train = np.expand_dims(X_train, axis=-1)
    X_val = np.expand_dims(X_val, axis=-1)

In [ ]:
# Define CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
 # Create and train the model
input_shape = (64, 64, 1)
model = create_cnn_model(input_shape)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

    # Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_acc:.4f}')

Epoch 1/10
  37/1217 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6914 - loss: 0.7874

I0000 00:00:1720155966.040709     183 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720155966.062969     183 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1217/1217 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7711 - loss: 0.6344

W0000 00:00:1720155974.365588     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720155975.084044     185 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1217/1217 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7711 - loss: 0.6344 - val_accuracy: 0.7952 - val_loss: 0.5410
Epoch 2/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7874 - loss: 0.5466 - val_accuracy: 0.7989 - val_loss: 0.5166
Epoch 3/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7936 - loss: 0.5230 - val_accuracy: 0.8013 - val_loss: 0.5139
Epoch 4/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8024 - loss: 0.4916 - val_accuracy: 0.8021 - val_loss: 0.4993
Epoch 5/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8026 - loss: 0.4844 - val_accuracy: 0.7998 - val_loss: 0.5212
Epoch 6/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8034 - loss: 0.4676 - val_accuracy: 0.8013 - val_loss: 0.5103
Epoch 7/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8109 - loss: 0.4433 - val_accuracy: 0.7986 - val_loss: 0.5241
Epoch 8/10
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8166 - loss: 0.4283 - val_accura

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Generate predictions on validation set
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_val, axis=1)

# Print confusion matrix and classification report
print(confusion_matrix(y_true_classes, y_pred_classes))
print(classification_report(y_true_classes, y_pred_classes, target_names=['Normal/Mild', 'Moderate', 'Severe']))

102/305 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

W0000 00:00:1720156032.893172     185 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
[[7273  246   37]
 [1151  319   92]
 [ 284  155  175]]
              precision    recall  f1-score   support

 Normal/Mild       0.84      0.96      0.89      7556
    Moderate       0.44      0.20      0.28      1562
      Severe       0.58      0.29      0.38       614

    accuracy                           0.80      9732
   macro avg       0.62      0.48      0.52      9732
weighted avg       0.76      0.80      0.76      9732



In [ ]:
# Prepare submission
def prepare_submission(test_image_dir, model):
    test_files = []
    test_images = []

    for root, dirs, files in os.walk(test_image_dir):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                test_files.append(file_path)
                test_images.append(load_dicom_image(file_path))

    test_images = np.array(test_images)
    test_images = np.expand_dims(test_images, axis=-1)  # Add channel dimension
    predictions = model.predict(test_images)
    pred_classes = np.argmax(predictions, axis=1)

    severity_mapping_rev = {0: 'normal_mild', 1: 'moderate', 2: 'severe'}
    submission = []

    for file_path, pred in zip(test_files, pred_classes):
        # Extract study_id and instance_number from file path
        parts = file_path.split('/')
        study_id = parts[-3]
        series_id = parts[-2]
        instance_number = parts[-1].replace('.dcm', '')
        row_id = f'{study_id}_{series_id}_{instance_number}'

        submission.append({
            'row_id': row_id,
            'normal_mild': 1 if pred == 0 else 0,
            'moderate': 1 if pred == 1 else 0,
            'severe': 1 if pred == 2 else 0,
        })

    submission_df = pd.DataFrame(submission)
    return submission_df

# Update test_image_dir with the actual path
test_image_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'
submission_df = prepare_submission(test_image_dir, model)

# Save submission file
submission_df.to_csv('submission.csv', index=False)
print(submission_df.head())

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step
                   row_id  normal_mild  moderate  severe
0  44036939_3844393089_12            0         1       0
1  44036939_3844393089_18            1         0       0
2   44036939_3844393089_9            1         0       0
3  44036939_3844393089_22            1         0       0
4  44036939_3844393089_25            1         0       0


In [ ]:
submission_df

,row_id,normal_mild,moderate,severe
0,44036939_3844393089_12,0,1,0
1,44036939_3844393089_18,1,0,0
2,44036939_3844393089_9,1,0,0
3,44036939_3844393089_22,1,0,0
4,44036939_3844393089_25,1,0,0
...,...,...,...,...
92,44036939_2828203845_6,1,0,0
93,44036939_2828203845_16,1,0,0
94,44036939_2828203845_20,1,0,0
95,44036939_2828203845_3,1,0,0
